In [5]:
# Gerekli kütüphaneleri import ediyoruz.
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
# İstediğimiz Url'e request atıyoruz ve kaynaktan çektiğimiz veriye bakıyoruz.

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

url = 'https://www.arabam.com/ikinci-el'

soup = get_soup(url)
print(soup)


<!DOCTYPE html>

<html lang="tr">
<head>
<title>İkinci El Araba Fiyatları, Satılık 2. El Araba Modelleri arabam.com'da!</title>
<meta content="İkinci El Araba fiyatlarını, aradığınız tüm marka ve modelleri, arabam.com güvencesiyle incelemek için hemen tıklayın!" name="description"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta charset="utf-8"/>
<meta content="arabam://" property="al:ios:url"/>
<meta content="369111943" property="al:ios:app_store_id"/>
<meta content="Arabam" property="al:ios:app_name"/>
<meta content="arabam://https://www.arabam.com" property="al:android:url"/>
<meta content="com.dogan.arabam" property="al:android:package"/>
<meta content="Arabam" property="al:android:app_name"/>
<meta content="https://www.arabam.com/ikinci-el" property="al:web:url">
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=1" name="viewport">
<meta content="İkinci El Araba Fiyatları, Satılık 2. El Araba Modelleri arabam.com'da!" na

In [7]:

car_names = []
descriptions = []
years = []
prices = []
dates = []
locations = []

# Konum bilgisini çekme
for item in soup.find_all("td", {"class": "listing-text"}):
    fade_out_div = item.find("div", {"class": "fade-out-content-wrapper"})
    if fade_out_div:
        location_span = fade_out_div.find("span")
        if location_span:  # Check if span is found
            locations.append(location_span.text.strip())

# Tarih bilgisini çekme
for item in soup.find_all("td", {"class": "listing-text tac"}):
    date_div = item.find("div", {"class": "fade-out-content-wrapper"})
    if date_div:
        date_a = date_div.find("a")
        if date_a:
            dates.append(date_a.text.strip())

# Araba adı, açıklama ve yıl bilgisini çekme
for item in soup.find_all("td", {"class": ["listing-modelname pr", "horizontal-half-padder-minus pr", "listing-text"]}):
    if "listing-modelname" in item['class']:
        car_names.append(item.text.strip())
    elif "horizontal-half-padder-minus" in item['class']:
        descriptions.append(item.text.strip())
    elif "listing-text" in item['class'] and "tac" not in item['class']:
        if len(years) < len(car_names):
            years.append(item.text.strip())

# Fiyat bilgisini çekme
price_tags = soup.find_all("span", {"class": "db no-wrap listing-price"})
prices = [tag.text.strip() for tag in price_tags]

#Tüm listelerin aynı uzunlukta olmasını sağlamak için minimum uzunluğu belirleme
length = min(len(car_names), len(descriptions), len(years), len(prices), len(dates), len(locations))



In [8]:
# DataFrame oluşturma
df1 = pd.DataFrame({
    'Car Name': car_names[:length],
    'Description': descriptions[:length],
    'Year': years[:length],
    'Price': prices[:length],
    'Date': dates[:length],
    'Location': locations[:length]
})

In [9]:

df1

,Car Name,Description,Year,Price,Date,Location
0,Volkswagen VW CC 1.4 TSi,300 BİN PEŞİN 36 AY KREDİ | 2012 CC 1.4TSİ EXC...,2012,845.000 TL,15 Mart 2024,İstanbul
1,Subaru Forester 2.0 TD Sport,Galeriden 2016 Subaru Forester 2.0 TDI,2016,1.250.000 TL,15 Mart 2024,İstanbul
2,Volkswagen Golf 1.4 TSi Highline,400 BİN PEŞİN 36 AY KREDİ | 2013 GOLF 7R+U LED...,2013,925.000 TL,15 Mart 2024,İstanbul
3,Opel Mokka 1.2 T Elegance,HATASIZ BOYASIZ ARTI GARANTILI,2021,1.214.900 TL,15 Mart 2024,Erzurum
4,Fiat Egea 1.4 Fire Easy,ÖZSAĞLAM'dan 2020 Egea 1.4 LPG-Easy Plus-51bin...,2020,675.000 TL,15 Mart 2024,Konya
5,Renault Megane 1.6 Alize,Galeriden Renault Megane 1.6 Alize 2001 Model ...,2001,300.000 TL,15 Mart 2024,Gaziantep
6,Volkswagen Golf 1.6 TDi BlueMotion Comfortline,_MTN_GROUP*TAN_2015_VW_GOLF_1.6_TDİ_COMFORTLİN...,2015,875.000 TL,15 Mart 2024,İstanbul
7,Fiat Doblo Combi 1.6 Multijet Premio Plus,İLK RUHSAT SAHİBİNDEN FULL SERVİS BAKIMLI FULL...,2016,565.000 TL,15 Mart 2024,Muğla
8,Renault Clio 1.0 TCe Joy,HATASİZ BOYASİZ 1.0 TCE MOTOR OTOMATİK VİTES O...,2021,825.000 TL,15 Mart 2024,İstanbul
9,Renault Twingo 1.2 Pack,ÖZEL PLAKALI - KLİMALI - 1.2 TWİNGO BENZİN LPG,1997,220.000 TL,15 Mart 2024,Edirne


In [10]:
df1.to_csv('Araba_veri.csv', index=False)